In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [2]:
df = pickle.load(open('/Users/giyeonlee/Downloads/추천시스템/추천시스템_프로젝트/Project_Final/YogingaDataSet.pkl','rb'))

In [3]:
print(df.tail())

         Restaurant UserID                                               Menu  \
23204  원할머니보쌈-상암배달점  in**님    맛있게 매운 의성마늘떡맵쌈/1(보쌈 사이즈 선택(중（생육710g이상,수육500g）))   
23205       장미로타리크럽  jk**님                      김.두.삼（김치두부삼겹살）/1,문어 초무침과 소면/1   
23206       장미로타리크럽  ja**님                                 문어삼합/1(사이즈 선택(3人))   
23207       장미로타리크럽     손님  유자보쌈（보쌈＋보쌈김치＋부추무침）/1(사이즈 선택(반근（300g）)),문어 초무침과...   
23208       장미로타리크럽  wd**님                                   통다리 문어버터구이와 땅콩/1   

                                                  Review Total Taste Quantity  \
23204  맛과 양 모두 만족스럽습니다! 맛있게 잘 먹겠습니다~ 배달도 빠르고 포장도 너무 깔...     5     5        5   
23205  배달이 좀 걸린듯 하지만 통두부에 삼겹김치.문어소면 맛있게 잘먹어요.폭풍흡입으로 사...     4     5        5   
23206                    여러번 먹었는데 맛있어요 사진이 없네요ㅠㅠ 호다닥 먹어서     5     5        5   
23207  친구네 놀러가서 시켜봤는데 진짜 전메뉴가 다 맛있다고 하는 친구말이 맞는거 같네요!...     5     5        4   
23208  그거알아요? 여기 메뉴 다맛있다는거☆ 항상오는 서비스 홍합탕 돈주고 시켜야되는거 아...     5     5        5   

      Delivery           D

In [4]:
df.rename(columns={'Total':'Rating'},inplace = True)
df['Rating'] = pd.to_numeric(df['Rating'])
# 기존 데이터 사이즈
df.shape

(358132, 9)

In [5]:
n_users = len(df['UserID'].unique())
n_restaurants = len(df['Restaurant'].unique())

n_users, n_restaurants

(1309, 635)

In [6]:
# n개 이상의 리뷰를 남긴 유저와 가게만 남기기
def preprocessing(data, n):
    min_id = data['UserID'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['UserID'].isin(min_id)]

    min_res = data['Restaurant'].value_counts() >= n
    min_res = min_res[min_res].index.to_list()
    data = data[data['Restaurant'].isin(min_res)]

    return data

In [7]:
temp = df.copy()
for i in range(1,10):
    temp = preprocessing(temp, 10)

df = temp
# 조정된 사이즈
df.shape

(356174, 9)

In [8]:
df['Rating'].describe()

count    356174.000000
mean          4.739678
std           0.731131
min           0.000000
25%           5.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: Rating, dtype: float64

In [9]:
#user-restaurant utility matrix
#UserID = '손님' 제거
user_resaurant = df.pivot_table('Rating',index='UserID',columns = 'Restaurant',aggfunc='max')
user_resaurant.drop(user_resaurant.loc[['손님'],:].index, inplace = True)
print(user_resaurant.shape)

#restaurant-restaurant utility matrix
retaurant_user = df.pivot_table('Rating',index='Restaurant',columns = 'UserID',aggfunc='max')
retaurant_user.drop(retaurant_user[['손님']], inplace = True, axis = 1)
print(retaurant_user.shape)



(884, 562)
(562, 884)


In [10]:
#결측치 0으로 대체
retaurant_user.fillna(0,inplace = True)
user_resaurant.fillna(0,inplace = True)

print(retaurant_user.head())
print(user_resaurant.head())

UserID                -_**님  -b**님  -e**님  00**님  01**님  02**님  03**님  04**님  \
Restaurant                                                                     
                 5.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
10평파스타&화덕피자      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1인기사식당           0.0    0.0    0.0    0.0    0.0    5.0    0.0    0.0    0.0   
1인용마라탕-서울수색점     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1인용묵은지김치찜-서울수색점  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

UserID           05**님  ...  zo**님  zp**님  zq**님  zr**님  zs**님  zt**님  zu**님  \
Restaurant              ...                                                    
                   0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
10평파스타&화덕피자        0.0  ...    0.0    0.0    5.0    0.0    0.0    0.0    0.0   
1인기사식당             0.0  ...    0.0    0.0    0.0    0.0    5.0    0.0    0.0   
1인용마라탕-서울수색점       0.0  ...    0.0    0

In [11]:
# 코사인 유사도 구하기
item_sim = cosine_similarity(retaurant_user, retaurant_user)

# cosine_similarity()로 반환된 넘파이 행렬을 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=user_resaurant.columns,
                          columns=user_resaurant.columns)

item_sim_df.head(3)

Restaurant,,10평파스타&화덕피자,1인기사식당,1인용마라탕-서울수색점,1인용묵은지김치찜-서울수색점,1인찌개와탕&냉면,1인한솥명가갈비찜-월곡점,1인할매보쌈족발-월곡점,24시 진짜루,5959양꼬치&마라탕,...,홍콩반점-합정점,홍콩할매불닭발-당산점,홍푸드&갈비찜&냉면,홍푸드1인분&갈비찜,황가네철판곱창전골,회대장산지직송숙성활어전문점,회뜨는총각-연남점,후라이드참잘하는집-마포점,희야네반찬,힐링스시-본점
Restaurant,,,,,,,,,,,,,,,,,,,,,
,1.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10평파스타&화덕피자,0.0,1.00000,0.50958,0.269001,0.288870,0.194043,0.421356,0.217198,0.509677,0.090425,...,0.370919,0.195720,0.515304,0.483801,0.510888,0.326947,0.492747,0.490072,0.301056,0.209885
1인기사식당,0.0,0.50958,1.00000,0.219170,0.310192,0.249844,0.435606,0.224409,0.586505,0.084991,...,0.358051,0.214618,0.575620,0.556899,0.584972,0.359635,0.540863,0.568616,0.308116,0.213054


In [12]:
# 예시로 '10평파스타&화덕피자' 가게와 코사일 유사도가 높은순으로 5개의 가게를 뽑아냄
item_sim_df['10평파스타&화덕피자'].sort_values(ascending=False)[:5]

Restaurant
10평파스타&화덕피자       1.000000
북촌손만두-홍익대점        0.553880
호식이두마리치킨-성산1호점    0.546064
옐로우피자-연남점         0.545110
마깐마포깐풍기-신촌본점      0.542192
Name: 10평파스타&화덕피자, dtype: float64

In [13]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)]) # 평점, 가게 유사도
    return ratings_pred

# 유저별 예측 평점 구하기
# 평점 value와 유사도 value만 뽑아서 대입
ratings_pred = predict_rating(user_resaurant.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=user_resaurant.index,
                                  columns = user_resaurant.columns)

# 유저별 계산된 예측 평점
ratings_pred_matrix

Restaurant,,10평파스타&화덕피자,1인기사식당,1인용마라탕-서울수색점,1인용묵은지김치찜-서울수색점,1인찌개와탕&냉면,1인한솥명가갈비찜-월곡점,1인할매보쌈족발-월곡점,24시 진짜루,5959양꼬치&마라탕,...,홍콩반점-합정점,홍콩할매불닭발-당산점,홍푸드&갈비찜&냉면,홍푸드1인분&갈비찜,황가네철판곱창전골,회대장산지직송숙성활어전문점,회뜨는총각-연남점,후라이드참잘하는집-마포점,희야네반찬,힐링스시-본점
UserID,,,,,,,,,,,,,,,,,,,,,
,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
-_**님,0.0,0.236436,0.248217,0.213629,0.226008,0.221633,0.227302,0.232752,0.256974,0.219362,...,0.237006,0.200346,0.242698,0.248529,0.253631,0.235727,0.245817,0.289605,0.219900,0.211341
-b**님,0.0,0.110789,0.112862,0.105436,0.115828,0.107204,0.104164,0.106094,0.112632,0.098320,...,0.107337,0.108660,0.109327,0.111413,0.110330,0.147663,0.110532,0.115566,0.115118,0.106167
-e**님,0.0,0.203754,0.220259,0.192645,0.190329,0.194376,0.201259,0.220295,0.224332,0.197091,...,0.207804,0.178384,0.215502,0.216998,0.220981,0.199969,0.218287,0.251726,0.200689,0.185571
00**님,0.0,0.550877,0.576029,0.531058,0.522490,0.536406,0.545779,0.542956,0.593053,0.511076,...,0.558560,0.488304,0.571474,0.575104,0.581748,0.538076,0.598311,0.635347,0.532634,0.526139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zt**님,0.0,0.062663,0.067104,0.062402,0.060852,0.063445,0.062197,0.062095,0.067365,0.061915,...,0.061993,0.055199,0.066381,0.076824,0.065512,0.061445,0.064955,0.073415,0.063136,0.056615
zu**님,0.0,0.606937,0.640663,0.607562,0.589474,0.591738,0.605792,0.600880,0.654021,0.572039,...,0.620174,0.556497,0.637835,0.639882,0.642555,0.590994,0.635371,0.704707,0.594546,0.600057
zx**님,0.0,1.025360,1.092341,0.998777,0.982955,0.982643,1.033257,1.042510,1.113803,0.930911,...,1.030418,0.932814,1.076353,1.073201,1.101561,0.990992,1.068858,1.197678,0.986337,0.951792


In [14]:
# 위에서 나온 유저-가게별 평점 예측표를 기반으로 mse를 통해 예측 정확성을 분석함
# mse가 높기때문에 그다지 좋지 않음
def get_mse(pred, actual):
    # 실제 평점이 있는 가게만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)
print('MSE: ', get_mse(ratings_pred, user_resaurant.values))

MSE:  9.966770847183565


In [17]:
# 아래의 username에 유저 이름을 넣어 해당 유저에게 추천하는 식당 반환
username='-_**님'

# 사용자의 예측 평점이 높은 가게 중, 사용자가 실제로 평점을 메기지 않은(아직 안먹어본) 가게를 반환
def non_visit_restaurant(ratings_matrix, username):
    # username로 입력받은 사용자의 모든 식당 정보를 추출해 Series로 반환
    # 반환된 user_rating은 Restaurant를 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[username, :]

    # user_rating이 0보다 크면 기존에 방문한 식당
    # 대상 인덱스를 추출해 list 객체로 만듦
    visited = user_rating[user_rating>0].index.tolist()

    # 모든 식당을 list 객체로
    restaurant_list = ratings_matrix.columns.tolist()

    # list comprehension으로 visited에 해당하는 식당은 restaurant_list에서 제외
    not_visited = [restaurant for restaurant in restaurant_list if restaurant not in visited]

    return not_visited

# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_visited에 들어온 식당 뽑기
# 가장 예측 평점이 높은 순으로 정렬
def recomm_restaurant_by_username(pred_df, userId, not_tried, top_n):
    recomm_restaurant = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_restaurant



# 예측 평점 데이터는 DataFrame으로 
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=user_resaurant.index,
                                   columns=user_resaurant.columns)

# 유저가 방문하지 않은 식당 추출
not_tried = non_visit_restaurant(user_resaurant, username)

# 상위 3개의 식당 추천
recomm_restaurant = recomm_restaurant_by_username(ratings_pred_matrix, username, not_tried, top_n=10)
recomm_restaurant = pd.DataFrame(data=recomm_restaurant.values, index=recomm_restaurant.index,
                           columns=['predict rate'])
recomm_restaurant

,predict rate
Restaurant,
꽃게나라 간장게장,0.306709
두남자의볶음밥＆포슈아베트남쌀국수,0.304825
대족장,0.292403
치킨플러스-성산점,0.285083
족발야시장-신촌점,0.284898
또래오래-마포망원점,0.278045
스시앤-마포점,0.275958
벚꽃스시-본점,0.274341
KFC-홍익대점,0.273157
